# W5E5 download and postprocessing

- from: https://data.isimip.org/search/query/w5e5/climate_forcing/w5e5v2.0/climate_variable/pr/climate_variable/orog/climate_variable/tas/
- download in terminal: `wget -i download_W5E5_script.txt` 


In [2]:
import xarray as xr
import numpy as np
import pandas as pd
from oggm import utils

import sys
import matplotlib.pyplot as plt
import os

path='/media/share/datasets/w5e5/'


ds_inv = xr.open_dataset(path + 'orog_W5E5v2.0.nc')
ds_inv = ds_inv.rename({'lat': 'latitude'})
ds_inv = ds_inv.rename({'lon': 'longitude'})
ds_inv.coords['longitude'] = np.where(ds_inv.longitude.values < 0,
                                      ds_inv.longitude.values + 360,
                                      ds_inv.longitude.values)
# (ds.coords['longitude'] + 180) % 360 - 180
ds_inv = ds_inv.sortby(ds_inv.latitude) # because it starts at 90 ... 

ds_inv = ds_inv.sortby(ds_inv.longitude)

ds_inv.longitude.attrs['units'] = 'degrees_onlypositive'

# get the dataset where coordinates of glaciers are stored
frgi = utils.file_downloader('https://cluster.klima.uni-bremen.de/~oggm/rgi/rgi62_stats.h5')
odf = pd.read_hdf(frgi, index_col=0)

nx, ny = ds_inv.dims['longitude'], ds_inv.dims['latitude']
# Nearest neighbor lookup
cenlon_for_bins = np.where(odf['CenLon'] < -0.125,
                           odf['CenLon']+360,
                           odf['CenLon']) # just make them into 0-> 360 scheme
lon_bins = np.linspace(0, 360, nx) # np.linspace(-0.125, 359.75+0.125, nx)
# !!! attention W5E5 sorted from 90 to -90 !!!!
#lat_bins = np.linspace(90, -90, ny)  #  
#lat_bins = np.linspace(90+0.125, -90-0.125, ny)
lat_bins = np.linspace(-90, +90, ny)
odf['lon_id'] = np.digitize(cenlon_for_bins, lon_bins)-1
odf['lat_id'] = np.digitize(odf['CenLat'], lat_bins)-1
# Use unique grid points as index and compute the area per location
odf['unique_id'] = ['{:03d}_{:03d}'.format(lon, lat) for (lon, lat) in zip(odf['lon_id'], odf['lat_id'])]
mdf = odf.drop_duplicates(subset='unique_id').set_index('unique_id')
mdf['Area'] = odf.groupby('unique_id').sum()['Area']
print('Total number of glaciers: {} and number of W5E5 gridpoints with glaciers in them: {}'.format(len(odf), len(mdf)))

# this is the mask that we need to remove all non-glacierized gridpoints
mask = np.full((ny, nx), np.NaN)
mask[mdf['lat_id'], mdf['lon_id']] = 1#mdf['Area']
ds_inv['glacier_mask'] = (('latitude', 'longitude'), np.isfinite(mask))

#c = (ds_inv.longitude - lon)**2 + (ds_inv.latitude - lat)**2
lon, lat = (10.7584, 46.8003)
surf_hef = ds_inv.sel(longitude=lon, latitude = lat, method='nearest')
np.testing.assert_allclose(surf_hef.orog.values, 2250, rtol=0.1)



Total number of glaciers: 216502 and number of W5E5 gridpoints with glaciers in them: 4713


/tmp/ipykernel_14618/3570220549.py:45: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mdf['Area'] = odf.groupby('unique_id').sum()['Area']


In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from oggm import utils

import sys
import matplotlib.pyplot as plt
import os

path='/home/www/lschuster/W5E5'


ds_inv = xr.open_dataset('orog_W5E5v2.0.nc')
#ds_inv = xr.open_dataset('/home/users/lschuster/glacierMIP/ASurf_WFDE5_CRU_v1.1.nc')
ds_inv = ds_inv.rename({'lat': 'latitude'})
ds_inv = ds_inv.rename({'lon': 'longitude'})
ds_inv.coords['longitude'] = np.where(ds_inv.longitude.values < 0,
                                      ds_inv.longitude.values + 360,
                                      ds_inv.longitude.values)
# (ds.coords['longitude'] + 180) % 360 - 180
ds_inv = ds_inv.sortby(ds_inv.latitude) # because it starts at 90 ... 

ds_inv = ds_inv.sortby(ds_inv.longitude)

ds_inv.longitude.attrs['units'] = 'degrees_onlypositive'

# get the dataset where coordinates of glaciers are stored
frgi = utils.file_downloader('https://cluster.klima.uni-bremen.de/~oggm/rgi/rgi62_stats.h5')
odf = pd.read_hdf(frgi, index_col=0)

nx, ny = ds_inv.dims['longitude'], ds_inv.dims['latitude']
# Nearest neighbor lookup
cenlon_for_bins = np.where(odf['CenLon'] < -0.125,
                           odf['CenLon']+360,
                           odf['CenLon']) # just make them into 0-> 360 scheme
lon_bins = np.linspace(0, 360, nx) # np.linspace(-0.125, 359.75+0.125, nx)
# !!! attention W5E5 sorted from 90 to -90 !!!!
#lat_bins = np.linspace(90, -90, ny)  #  
#lat_bins = np.linspace(90+0.125, -90-0.125, ny)
lat_bins = np.linspace(-90, +90, ny)
odf['lon_id'] = np.digitize(cenlon_for_bins, lon_bins)-1
odf['lat_id'] = np.digitize(odf['CenLat'], lat_bins)-1
# Use unique grid points as index and compute the area per location
odf['unique_id'] = ['{:03d}_{:03d}'.format(lon, lat) for (lon, lat) in zip(odf['lon_id'], odf['lat_id'])]
mdf = odf.drop_duplicates(subset='unique_id').set_index('unique_id')
mdf['Area'] = odf.groupby('unique_id').sum()['Area']
print('Total number of glaciers: {} and number of W5E5 gridpoints with glaciers in them: {}'.format(len(odf), len(mdf)))

# this is the mask that we need to remove all non-glacierized gridpoints
mask = np.full((ny, nx), np.NaN)
mask[mdf['lat_id'], mdf['lon_id']] = 1#mdf['Area']
ds_inv['glacier_mask'] = (('latitude', 'longitude'), np.isfinite(mask))

#c = (ds_inv.longitude - lon)**2 + (ds_inv.latitude - lat)**2
lon, lat = (10.7584, 46.8003)
surf_hef = ds_inv.sel(longitude=lon, latitude = lat, method='nearest')
np.testing.assert_allclose(surf_hef.orog.values, 2250, rtol=0.1)



In [ ]:
#ds_inv_wfde5 = xr.open_dataset('/home/users/lschuster/ASurf_WFDE5_CRU_v1.1.nc')

## yearly and monthly (actually both together ...)


In [ ]:

make_daily = False
if make_daily:
    
    try:
        os.mkdir('/home/users/lschuster/W5E5/flattened')
        os.mkdir('/home/users/lschuster/W5E5/flattened/tmp_pr')
        os.mkdir('/home/users/lschuster/W5E5/flattened/tmp_tas')
    except:
        pass
    
    yss = [1979, 1981, 1991, 2001, 2011]
    yee = [1980, 1990, 2000, 2010, 2019]
    for var in ['pr', 'tas']:
        for ys, ye in zip(yss, yee):
            ds = xr.open_dataset('{}_W5E5v2.0_{}0101-{}1231.nc'.format(var, ys, ye))

            ds = ds.rename({'lon':'longitude'}).rename({'lat':'latitude'})
            ds.coords['longitude'] = np.where(ds.longitude.values < 0,
                                              ds.longitude.values + 360,
                                              ds.longitude.values)

            ds = ds.sortby(ds.longitude)
            ds.longitude.attrs['units'] = 'degrees_onlypositive'

            # this takes too long !!!
            # ds_merged_glaciers = xr_prcp.where(ds_inv['glacier_mask'], drop = True)

            ds['ASurf'] = ds_inv['orog'] #ds_inv['ASurf']
            if ys==1979 and var =='pr':

                # as we use here only those gridpoints where glaciers are involved, need to put the mask on dsi as well!
                # dsi = ds_inv.where(ds_inv['glacier_mask'], drop = True)  # this makes out of in total 6483600 points only 116280 points!!!
                dsi = ds.isel(time=[0]).where(ds_inv['glacier_mask'], drop = True)
                # we do not want any dependency on latitude and longitude
                dsif = dsi.stack(points=('latitude', 'longitude')).reset_index(('points'))
                #dsif # so far still many points 

                # drop the non-glacierized points
                dsifs = dsif.where(np.isfinite(dsi[var].stack(points=('latitude',
                                                                      'longitude')).reset_index(('time',
                                                                                                 'points'))), drop=True)
                # I have to drop the 'time_' dimension, to be equal to the era5_land example, because the invariant file should not have any time dependence !
                dsifs = dsifs.drop_vars(var)
                dsifs = dsifs.drop('time')
                dsifs = dsifs.drop('time_')
                dsifs.to_netcdf('/home/users/lschuster/W5E5/flattened/w5e5v2.0_glacier_invariant_flat.nc')

                #check if gridpoint nearest to hef is right!!!
                # happened once that something went wrong here ...
                lon, lat = (10.7584, 46.8003)
                #orog = xr.open_dataset('/home/lilianschuster/Downloads/w5e5v2.0_glacier_invariant_flat.nc').ASurf
                orog = dsifs.ASurf
                c = (orog.longitude - lon)**2 + (orog.latitude - lat)**2
                surf_hef = orog.isel(points=c.argmin())
                np.testing.assert_allclose(surf_hef, 2250, rtol=0.1)
            ds = ds.drop_vars('ASurf')

            for t in ds.time.values[:]:  # ds_merged_glaciers.time.values: .sel(time=slice('1901-04','2016-12'))
                # produce a temporary file for each month
                sel_l = ds.sel(time=[t])
                # don't do the dropping twice!!!!
                #sel = sel_l.where(ds_inv['glacier_mask'], drop = True)
                sel = sel_l.where(ds_inv['glacier_mask'])
                sel = sel.stack(points=('latitude', 'longitude')).reset_index(('time', 'points'))
                sel = sel.where(np.isfinite(sel[var]), drop=True)
                sel.to_netcdf('/home/users/lschuster/W5E5/flattened/tmp_{}/tmp_{}.nc'.format(var, str(t)[:10]))


In [1]:
import xarray as xr
import numpy as np
# make yearly files
var = 'pr'
y = 1980
dso_y = xr.open_mfdataset('/home/bsurya/work/oggm/AIR-suitability-index/MBsandbox/flattened/tmp_{}/tmp_{}-*.nc'.format(var, str(y)),
                                  concat_dim='time', combine='nested') # .rename_vars({'time_':'time'})
print(dso_y)
dso_y.to_netcdf('/home/users/lschuster/W5E5/flattened/tmp_{}/tmp2_{}.nc'.format(var, str(y)))
dso_y.close()

<xarray.Dataset>
Dimensions:    (time: 366, points: 4713)
Coordinates:
  * time       (time) datetime64[ns] 1980-01-01 1980-01-02 ... 1980-12-31
    latitude   (points) float64 dask.array<chunksize=(4713,), meta=np.ndarray>
    longitude  (points) float64 dask.array<chunksize=(4713,), meta=np.ndarray>
Dimensions without coordinates: points
Data variables:
    pr         (time, points) float32 dask.array<chunksize=(1, 4713), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.7
    title:        WFDE5 over land merged with ERA5 over the ocean (W5E5) vers...
    institution:  Potsdam Institute for Climate Impact Research (PIK)
    project:      Inter-Sectoral Impact Model Intercomparison Project phase 3...
    contact:      ISIMIP cross-sectoral science team <info@isimip.org> <https...
    summary:      WFDE5 (with GPCC precipitation correction) over land merged...
    references:   Cucchi et al. (2020) <https://doi.org/10.5194/essd-12-2097-...
    version:      2.0


PermissionError: [Errno 13] Permission denied: b'/home/users/lschuster/W5E5/flattened/tmp_pr/tmp2_1980.nc'

In [18]:
import xarray as xr
import numpy as np
# make yearly files
for var in ['pr']:
    for y in np.arange(1979, 2020):
        print(var,y)
        dso_y = xr.open_mfdataset('/home/bsurya/work/oggm/AIR-suitability-index/MBsandbox/flattened/tmp_{}/tmp_{}-01-*.nc'.format(var, str(y)),
                                  concat_dim='time', combine='nested') # .rename_vars({'time_':'time'})
        print(dso_y)
        dso_y.to_netcdf('/home/users/lschuster/W5E5/flattened/tmp_{}/tmp2_{}.nc'.format(var, str(y)))
        dso_y.close()
    
# aggregate yearly files
for var in ['pr', 'tas']:
    dso_all2 = xr.open_mfdataset('/home/users/lschuster/W5E5/flattened/tmp_{}/tmp2_*.nc'.format(var),
                        concat_dim='time', combine='nested', parallel = True).rename_vars({'time_':'time'})
    
    dso_all2.attrs['history'] = 'longitudes to 0 -> 360,  only glacier gridpoints chosen and flattened latitude/longitude --> points'
    dso_all2.attrs['postprocessing_date'] = str(np.datetime64('today','D'))
    dso_all2.attrs['postprocessing_scientist'] = 'lilian.schuster@student.uibk.ac.at'
    dso_all2.to_netcdf('/home/users/lschuster/W5E5/flattened/w5e5v2.0_{}_global_daily_flat_glaciers_1979_2019.nc'.format(var))
    




pr 1979
<xarray.Dataset>
Dimensions:    (time: 31, points: 4713)
Coordinates:
  * time       (time) datetime64[ns] 1979-01-01 1979-01-02 ... 1979-01-31
    latitude   (points) float64 dask.array<chunksize=(4713,), meta=np.ndarray>
    longitude  (points) float64 dask.array<chunksize=(4713,), meta=np.ndarray>
Dimensions without coordinates: points
Data variables:
    pr         (time, points) float32 dask.array<chunksize=(1, 4713), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.7
    title:        WFDE5 over land merged with ERA5 over the ocean (W5E5) vers...
    institution:  Potsdam Institute for Climate Impact Research (PIK)
    project:      Inter-Sectoral Impact Model Intercomparison Project phase 3...
    contact:      ISIMIP cross-sectoral science team <info@isimip.org> <https...
    summary:      WFDE5 (with GPCC precipitation correction) over land merged...
    references:   Cucchi et al. (2020) <https://doi.org/10.5194/essd-12-2097-...
    version:      2.0


PermissionError: [Errno 13] Permission denied: b'/home/users/lschuster/W5E5/flattened/tmp_pr/tmp2_1979.nc'

In [ ]:
# make monthly files 
for var in ['pr', 'tas']:

    pathi= 'flattened/w5e5v2.0_{}_global_daily_flat_glaciers_1979_2019.nc'.format(var)
    ds = xr.open_dataset(pathi)


    ds_monthly = ds.resample(time='MS', keep_attrs=True).mean(keep_attrs=True)

    ds_monthly.attrs['postprocessing_scientist'] = 'lilian.schuster@student.uibk.ac.at'
    ds_monthly.attrs['postprocessing_actions'] =  ("using xarray: ds_monthly = ds.resample(time='MS', keep_attrs=True).mean(keep_attrs=True)\n"
                                                                      "ds_monthly.to_netcdf()\n")

    ds_monthly.to_netcdf('flattened/w5e5v2.0_{}_global_monthly_flat_glaciers_1979_2019.nc'.format(var))
    
    if var == 'tas':
        # also compute monthly daily std:

        ds_tas_daily_std = ds.resample(time='MS', keep_attrs=True).std(keep_attrs=True)
        ds_tas_daily_std = ds_tas_daily_std.rename_vars(dict(tas='tas_std'))
        # now have to change variable tas to tas_std and its attributes 
        ds_tas_daily_std.tas_std.attrs['standard_name'] = 'air_temperature_daily_std'
        ds_tas_daily_std.tas_std.attrs['long_name'] = 'Near-Surface Air Temperature daily standard deviation'
        ds_tas_daily_std.attrs['postprocessing_date'] = str(np.datetime64('today','D'))
        ds_tas_daily_std.attrs['postprocessing_scientist'] = 'lilian.schuster@student.uibk.ac.at'
        ds_tas_daily_std.attrs['postprocessing_actions'] =  ("using xarray: \n"
                                                                      "ds_tas_daily_std = ds.resample(time='MS', keep_attrs=True).std(keep_attrs=True)\n"
                                                                       "ds_tas_daily_std = ds_tas_daily_std.rename_vars(dict(tas='tas_std'))\n"
                                                                       "ds_tas_daily_std.tas_std.attrs['standard_name'] = 'air_temperature_daily_std'\n"
                                                                       "ds_tas_daily_std.tas_std.attrs['long_name'] = 'Near-Surface Air Temperature daily standard deviation'\n"
                                                                      "ds_tas_daily_std.to_netcdf(...)\n")
        
        ds_tas_daily_std.to_netcdf('flattened/w5e5v2.0_tas_std_global_monthly_flat_glaciers_1979_2019.nc')

In [ ]:
#import os
#os.remove('/home/users/lschuster/glacierMIP/isimip3b/flattened/tmp_monthly_{}/'.format(var))
import glob
for var in ['pr', 'tas']:
    files = glob.glob('/home/users/lschuster/W5E5/flattened/tmp_{}/tmp*'.format(var))
    for f in files:
        os.remove(f)

In [ ]:
pr = xr.open_dataset('flattened/w5e5v2.0_pr_global_daily_flat_glaciers_1979_2019.nc')

In [ ]:
tas = xr.open_dataset('flattened/w5e5v2.0_tas_global_daily_flat_glaciers_1979_2019.nc')

In [ ]:
lon, lat = (10.7584, 46.8003)
inv = xr.open_dataset('flattened/w5e5v2.0_glacier_invariant_flat.nc').ASurf
c = (inv.longitude - lon)**2 + (inv.latitude - lat)**2
tas_sel = tas.isel(points=c.argmin())
pr_sel = pr.isel(points=c.argmin())

tas_sel.tas


In [ ]:
plt.plot(pr_sel.pr)

In [ ]:
tas_sel.tas.mean()

In [ ]:
plt.plot(tas_sel.tas)

**move the files somewhere where everyone can get them**
- `mkdir /home/www/lschuster/w5e5v2.0`
- `mkdir /home/www/lschuster/w5e5v2.0/flattened`
- `mkdir /home/www/lschuster/w5e5v2.0/flattened/monthly`
- `mkdir /home/www/lschuster/w5e5v2.0/flattened/daily`
- `cp flattened/w5e5*monthly* /home/www/lschuster/w5e5v2.0/flattened/monthly/`
- `cp flattened/w5e5*daily* /home/www/lschuster/w5e5v2.0/flattened/daily/`
- `cp flattened/w5e5*inv* /home/www/lschuster/w5e5v2.0/flattened/daily/`
- `cp flattened/w5e5*inv* /home/www/lschuster/w5e5v2.0/flattened/monthly/`